In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.agents.a3c.a3c_tf_policy import A3CTFPolicy
from ray.rllib.agents.a3c import a3c
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d import MyEnv
#from test_env_for_lstm import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions
#from modules.models import MyConv2DModel_v0B_Small_CBAM_1DConv_Share
#from modules.models import MyRNNUAVClass
#from modules.models import DenseNetModelLarge
from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
from utility.save_logs import save_logs
from utility.save_logs import save_hists
from utility.save_logs import save_env_info

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 1
CONTINUAL = False
NUM_EVAL = 1
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

#ModelCatalog.register_custom_model('my_model', MyRNNUAVClass)

eval_env = MyEnv()
policies = {
    #"blue_1": PolicySpec(config={"gamma": 0.99}),
    #"blue_2": PolicySpec(config={"gamma": 0.95}),
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},}),
    #"blue_0": (A3CTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    #"blue_1": (A3CTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
}
policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env
config = ppo.DEFAULT_CONFIG.copy()
config = {"env": MyEnv,"num_gpus": 0,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 0,
          "train_batch_size": 600*5*10,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          #"clip_actions":True,"normalize_actions":True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "explore":True,
          "sgd_minibatch_size": 300, "num_sgd_iter":20,
          "exploration_config": {"type": "StochasticSampling","random_timesteps":0}, #PPO デフォルト "random_timesteps":0
          #"model":{"fcnet_activation": "relu","fcnet_hiddens": [256, 256, 256],"post_fcnet_activation": "linear",
          #         "vf_share_layers": True,},#"linear","relu","tanh" "use_lstm":True,"lstm_cell_size":256,"max_seq_len":128
          
          #"model": {"custom_model": "my_model"},
          "multiagent": {"policies": policies,  "policy_mapping_fn": policy_mapping_fn}
         }
#res_name = "sgd"+str(config["sgd_minibatch_size"])+"sgd_num"+str(config["num_sgd_iter"])+"lr"+str(config["lr"])+"gamma"+str(config["gamma"])
res_name = "test"
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    trainer.restore(checkpoint_path=model_path)

models_dir = os.path.join('./' + PROJECT + '/models/')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
for j in range(2):
    text_name = models_dir + TRIAL + "blue_"+str(j) +'.txt'
    with open(text_name, "w") as fp:
        trainer.get_policy("blue_"+str(j)).model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
    png_name = models_dir + TRIAL + '.png'
    plot_model(trainer.get_policy("blue_"+str(j)).model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):
C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
2022-03-22 05:24:14,100	INFO trainer.py:723 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also want to then set `eager_tracing=True` in order to reach similar execution speed as with static-graph mode.
2022-03-22 05:24:14,101	INFO ppo.py:167 -- In multi-agent mode, policies will be optimized sequentially by the mult

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar  7 21:50:01 2022

@author: Takumi
"""
eval_env.reset()
save_env_info(eval_env)
record_mode = 0
results_dir = os.path.join('./' + PROJECT + '/results/')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.eval = False
    eval_env.reset()
    results = trainer.train()
    save_logs(res_name,results,steps,CONTINUAL)
    print(pretty_print(results))
    #check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n-------------- Evaluation at steps:{steps} starting ! --------------')

        check_point = trainer.save(checkpoint_dir=check_point_dir)
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
            trainer.restore(checkpoint_path=model_path)
            eval_env.eval = True
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            #fig = plt.figure(1,figsize=(8.0, 6.0))
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100

            cell_size = 256
            state_0=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            state_1=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            action_dict0 = [0,0]
            action_dict1 = [0,0]
            rewards = {"blue_0":0,"blue_1":0}
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                #video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

            while True:
                action_dict = {}
                action_dict0 = trainer.compute_single_action(obs['blue_0'],
                                                             state=state_0,prev_action=None,prev_reward=None,
                                                             policy_id='blue_0',explore=False)
                action_dict1 = trainer.compute_single_action(obs['blue_1'],
                                                             state=state_1,prev_action=None,prev_reward=None,
                                                             policy_id='blue_1',explore=False)
                
                #action_dict0 = trainer.compute_single_action(obs['blue_0'],policy_id='blue_0')
                #action_dict1 = trainer.compute_single_action(obs['blue_1'],policy_id='blue_1')
                state_0 = action_dict0[1]
                state_1 = action_dict1[1]
                obs, rewards, dones, infos = eval_env.step({'blue_0': action_dict0[0], 'blue_1': action_dict1[0]})

                env_blue_pos_temp_mod, env_red_pos_temp_mod, env_mrm_pos_temp_mod = render_env.copy_from_env_mod(eval_env)
                if eval_env.timer == 1:
                    env_blue_pos_mod = env_blue_pos_temp_mod
                    env_red_pos_mod = env_red_pos_temp_mod
                    env_mrm_pos_mod = env_mrm_pos_temp_mod
                else:
                    env_blue_pos_mod = np.vstack([env_blue_pos_mod,env_blue_pos_temp_mod])
                    env_red_pos_mod = np.vstack([env_red_pos_mod,env_red_pos_temp_mod])
                    env_mrm_pos_mod = np.vstack([env_mrm_pos_mod,env_mrm_pos_temp_mod])

                # plt.clf()

                # plt.subplots_adjust(left=-0.1,right=1.1,bottom=-0.1,top=1.1)
                # fig.canvas.draw()
                # plt.pause(.01)

                #if record_mode == 0:
                    #img = np.array(fig.canvas.renderer.buffer_rgba())
                    #img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    #video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()
                    
                #env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp = render_env.copy_from_env(eval_env)
                
                #env_blue_pos.append(env_blue_pos_temp)
                #env_red_pos.append(env_red_pos_temp)
                #env_mrm_pos.append(env_mrm_pos_temp)
                #step_num = step_num + 1
                # エピソードの終了処理
                if dones['__all__']:
                    save_hists("blue",steps,env_blue_pos_mod)
                    save_hists("red",steps,env_red_pos_mod)
                    save_hists("mrm",steps,env_mrm_pos_mod)
                    # print(f'all done at {env.steps}')
                    break
                

            
            #if record_mode == 0:
               # video.release()

ray.shutdown()

-------------------------- Scene: 0 --------------------------

----------------- Training at steps:0 start! -----------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
182 blue_0 DOWN
390 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 390 -0.10100165929032091 -2.6279335492837923
blue_1 False False 390 -1.1012001926236543 -3.319634291230762
-------------------------- Scene: 0 --------------------------
47 blue_1 DOWN
75 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 75 -1.1010033642113868 -1.3047682891738999
blue_1 False False 75 -0.10100166421138682 -1.2039365046535004
-------------------------- Scene: 0 --------------------------
84 blue_1 DOWN
445 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 445 -1.101002537520496 -3.6609309200327877
blue_1 False False 445 -0.10100163752049585 -1.3049724664835212
-------------------------- Scene: 0 --------------------------
116 blue_1 DOWN
570 blue_0 DOWN
TIME LIM

101 blue_0 DOWN
414 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 414 -0.10100155786598229 -1.3543453712906925
blue_1 False False 414 -1.1009984578659824 -6.074985653691203
-------------------------- Scene: 0 --------------------------
46 blue_0 DOWN
66 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 66 -0.10100051616423147 -1.3050437891316797
blue_1 False False 66 -1.1010169494975648 -1.2042189757422246
-------------------------- Scene: 0 --------------------------
418 blue_1 DOWN
562 blue_0 Shoot at red_1 launch distance : 44078.04607123247 True True
575 red_1 Shoot at blue_0
Same tgt shoot
587 blue_0 Shoot at red_0 launch distance : 42186.295478129236 True True
594 red_0 Shoot at blue_0
620 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 620 0.36128542653149187 55.880439598903486
blue_1 False False 620 0.39900112653149183 -2.729277607473113
-------------------------- Scene: 0 --------------------------
70 blue_1 DOWN
369 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 369 -1.101

99 blue_1 DOWN
215 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 215 -1.1009994602777229 -3.620476835262317
blue_1 False False 215 -0.10100156027772282 -1.4174396158791627
-------------------------- Scene: 0 --------------------------
45 blue_0 DOWN
85 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 85 -0.10100166033414532 -1.3050032048957705
blue_1 False False 85 -1.1009982603341453 -1.3050508121651792
-------------------------- Scene: 0 --------------------------
71 blue_1 DOWN
275 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 275 -1.1010021856471353 -3.353480824558723
blue_1 False False 275 -0.10100161898046867 -1.3049261147029245
-------------------------- Scene: 0 --------------------------
218 blue_1 DOWN
227 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 227 -1.1009973287054637 -2.5802079005909295
blue_1 False False 227 -0.10100166203879707 -3.4952872215656607
-------------------------- Scene: 0 --------------------------
46 blue_0 DOWN
211 blue_1 DOWN
TIME LIMIT LOSE
blue_0 Fa

74 blue_0 DOWN
154 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 154 -0.10100179781080681 -1.305242137669562
blue_1 False False 154 -1.1010016978108068 -1.4421289592885496
-------------------------- Scene: 0 --------------------------
152 blue_0 DOWN
174 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 174 -0.1010016645939171 -1.4334433976712515
blue_1 False False 174 -1.100998564593917 -3.1731526831595214
-------------------------- Scene: 0 --------------------------
140 blue_0 DOWN
141 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 141 -0.11317612596250537 -3.067254525361836
blue_1 False False 141 -1.1009988259625054 -1.407394551406906
-------------------------- Scene: 0 --------------------------
58 blue_1 DOWN
169 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 169 -1.1010045676845444 -2.9840440688426186
blue_1 False False 169 -0.10100140101787775 -1.4073326963745665
-------------------------- Scene: 0 --------------------------
84 blue_1 DOWN
300 blue_0 DOWN
TIME LIMIT LOSE
blue_0 F

42 blue_0 DOWN
47 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 47 -0.10100166505400965 -1.303950614208413
blue_1 False False 47 -1.1010034317206763 -1.305058705193689
-------------------------- Scene: 0 --------------------------
126 blue_1 DOWN
138 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 138 -1.1010009984213698 -1.8182128363887644
blue_1 False False 138 -0.1010016650880365 -1.3280084985892489
-------------------------- Scene: 0 --------------------------
65 blue_0 DOWN
82 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 82 -0.10100050779754104 -1.4067694039483722
blue_1 False False 82 -1.1009940411308743 -1.4094820392908347
-------------------------- Scene: 0 --------------------------
49 blue_0 DOWN
95 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 95 -0.10100164607746855 -1.3049263224813925
blue_1 False False 95 -1.1010047794108018 -1.310046344525293
-------------------------- Scene: 0 --------------------------
110 blue_0 DOWN
340 blue_1 Shoot at red_1 launch distance : 5863

...

46 blue_1 DOWN
48 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 48 -1.1010001646976317 -1.3050312811999742
blue_1 False False 48 -0.10100166469763164 -1.3040873849436565
-------------------------- Scene: 0 --------------------------
50 blue_0 DOWN
111 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 111 -0.10100166423453778 -1.6090974907847704
blue_1 False False 111 -1.101000497567871 -1.203778837533297
-------------------------- Scene: 0 --------------------------
189 blue_0 DOWN
265 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 265 -0.10100166275527975 -2.0887294185818894
blue_1 False False 265 -1.101000896088613 -3.3950859241732947
-------------------------- Scene: 0 --------------------------
53 blue_1 DOWN
217 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 217 -1.1010020983861188 -1.3264661283314916
blue_1 False False 217 -0.10100166505278556 -1.3046590675285572
-------------------------- Scene: 0 --------------------------
291 blue_1 DOWN
430 blue_0 DOWN
TIME LIMIT LOSE
blue_0 Fa

2022-03-22 22:19:54,580	INFO trainable.py:468 -- Restored on 127.0.0.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_000016\checkpoint-16
2022-03-22 22:19:54,581	INFO trainable.py:475 -- Current state after restoring: {'_iteration': 16, '_timesteps_total': 0, '_time_total': 59631.17302823067, '_episodes_total': 1881}


agent_timesteps_total: 965572
custom_metrics: {}
date: 2022-03-22_22-19-54
done: false
episode_len_mean: 256.4188034188034
episode_media: {}
episode_reward_max: 163.7183642397474
episode_reward_mean: 8.868769978330786
episode_reward_min: -9.974945895525535
episodes_this_iter: 117
episodes_total: 1881
experiment_id: 0b5eca70ed9f49578d683a6877c2f6db
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 25.949268341064453
        cur_lr: 0.0002500000118743628
        entropy: 8.331482887268066
        entropy_coeff: 0.0
        kl: 0.010250214487314224
        model: {}
        policy_loss: 0.010733537375926971
        total_loss: 20.87412452697754
        vf_explained_var: -0.44192883372306824
        vf_loss: 20.5974063873291
    blue_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 25.949268341064453
        cur_lr: 0.0002500000118743628
        entropy: 8.510189056396484
        entropy_coeff: 0.0
     

128 blue_0 DOWN
163 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 163 -0.10100166420542954 -1.34988734634941
blue_1 False False 163 -1.1009981975387628 -2.186671744365662
-------------------------- Scene: 0 --------------------------
71 blue_1 DOWN
306 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 306 -1.1010033973481435 -2.981198464529004
blue_1 False False 306 -0.10100166401481017 -1.3049300903759335
-------------------------- Scene: 0 --------------------------
105 blue_0 DOWN
165 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 165 -0.101001661537102 -2.328232556475967
blue_1 False False 165 -1.1009986282037687 -2.5300410361922516
-------------------------- Scene: 0 --------------------------
92 blue_1 DOWN
208 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 208 -1.101004088922609 -3.1475320041431116
blue_1 False False 208 -0.10100132225594219 -1.4059266302667643
-------------------------- Scene: 0 --------------------------
131 blue_0 DOWN
486 blue_1 Shoot at red_0 launch distance 

68 blue_0 DOWN
85 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 85 -0.10100166085237382 -1.3050732585852556
blue_1 False False 85 -1.1010068941857072 -1.305439390715087
-------------------------- Scene: 0 --------------------------
69 blue_0 DOWN
102 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 102 -0.10100166234415942 -1.4060596944170272
blue_1 False False 102 -1.1010034623441594 -1.4061782293996323
-------------------------- Scene: 0 --------------------------
44 blue_0 DOWN
56 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 56 -0.101 -1.2039794088995368
blue_1 False False 56 -1.1010082333333333 -1.4060649063326043
-------------------------- Scene: 0 --------------------------
69 blue_0 DOWN
226 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 226 -0.10100164695447841 -1.3062133069332884
blue_1 False False 226 -1.1010051136211452 -1.3517445012890745
-------------------------- Scene: 0 --------------------------
43 blue_1 DOWN
113 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 113 -1.